In [19]:
import pandas as pd
import requests

df = pd.read_csv(r"C:\Users\claud\OneDrive\Escritorio\Ironhack\Labs\week 3\Usa-Shootings-and-State-Parties\gva_mass_shootings-2025-10-21.csv")
df

,incident_id,incident_date,incident_time,updated_date,state,city_or_county,address,business_location_name,latitude,longitude,...,injured,victims_killed,victims_injured,suspects_killed,suspects_injured,suspects_arrested,incident_characteristics,sources,date_fixed,year
0,809048,"April 1, 2017",NaN,1491348530,Florida,Tampa,219 S Howard Ave,Palace Lounge,27.9431,-82.4831,...,4,1,4,0,0,1,"Shot - Wounded/Injured, Shot - Dead (murder, a...","c(""http://wfla.com/2017/04/01/tampa-police-inv...",2017-04-01,2017
1,809499,"April 1, 2017",5:25 AM,1680156402,Oklahoma,Oklahoma City,1212 N Woodward Ave,NaN,35.4810,-97.5754,...,3,2,3,0,0,1,"Shot - Wounded/Injured, Shot - Dead (murder, a...","c(""http://www.news9.com/story/35055747/okc-pol...",2017-04-01,2017
2,809815,"April 1, 2017",NaN,1492995601,California,Pittsburg,100 block of Madoline Street,NaN,38.0053,-121.8950,...,5,0,5,0,0,1,"Shot - Wounded/Injured, Mass Shooting (4+ vict...",http://www.nbcbayarea.com/news/local/Shooting-...,2017-04-01,2017
3,1082835,"April 1, 2018",2:15 AM,1522693801,Michigan,Saginaw,600 block of Yale St,NaN,43.4192,-83.9819,...,5,0,5,0,0,0,"Shot - Wounded/Injured, Mass Shooting (4+ vict...",http://www.mlive.com/news/saginaw/index.ssf/20...,2018-04-01,2018
4,1083011,"April 1, 2018",NaN,1646889853,Alabama,Mobile,3556 Halls Mill Rd,Grand Hall,30.6387,-88.1228,...,4,1,4,0,0,3,"Shot - Wounded/Injured, Shot - Dead (murder, a...","c(""http://www.al.com/news/mobile/index.ssf/201...",2018-04-01,2018
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5463,2411007,"September 9, 2022",4:50 PM,1722738993,New York,Bronx,810 Astor Ave,Pelham Parkway Houses,40.8592,-73.8634,...,4,0,4,0,0,1,"Shot - Wounded/Injured, Mass Shooting (4+ vict...","c(""https://twitter.com/NYC_Alerts911/status/15...",2022-09-09,2022
5464,2411901,"September 9, 2022",11:50 PM,1689272621,Minnesota,Minneapolis,328 W Broadway Ave,4th Street Saloon,44.9994,-93.2848,...,3,1,3,0,0,0,"Shot - Wounded/Injured, Shot - Dead (murder, a...","c(""https://www.fox9.com/news/2-pregnant-women-...",2022-09-09,2022
5465,2410446,"September 9, 2022",9:00 AM,1715920831,Maryland,Elk Mills,47 Hebron Ct,NaN,39.6611,-75.8365,...,0,4,0,1,0,0,"Shot - Dead (murder, accidental, or suicide), ...","c(""https://www.cbsnews.com/baltimore/news/five...",2022-09-09,2022
5466,2703975,"September 9, 2023",11:30 PM,1694472936,South Carolina,Rock Hill,800 block of Brunswick Dr,NaN,34.8921,-81.0411,...,4,0,4,0,0,2,"Shot - Wounded/Injured, Mass Shooting (4+ vict...",https://www.heraldonline.com/news/local/crime/...,2023-09-09,2023


In [43]:
years=list(range(2014,2025))
def get_edu(year):
    url_edu = f"https://api.census.gov/data/{year}/acs/acs1"
    params = {
        "get": "NAME,B15003_001E,B15003_002E,B15003_003E,B15003_004E,B15003_005E,"
        "B15003_006E,B15003_017E,B15003_022E,B15003_023E,B15003_025E",
        "for": "state:*"}
    response_1 = requests.get(url_edu, params=params)
    if response_1.status_code != 200:
        print(f"⚠️ {year}: HTTP {response_1.status_code} – skipping year")
        return pd.DataFrame()
    if not response_1.text.strip().startswith('[['):
        print(f"⚠️ {year}: invalid or empty response – skipping year")
        return pd.DataFrame()

    try:
        data = response_1.json()
    except Exception:
        print(f"⚠️ {year}: cannot decode JSON – skipping year")
        return pd.DataFrame()

    data = response_1.json()
    df_edu = pd.DataFrame(data[1:], columns=data[0])
    df_edu = df_edu.rename(columns={
        "NAME": "State",
        "B15003_001E": "Population_25plus",
        "B15003_002E": "No_Schooling_Completed",
        "B15003_003E": "Nursery_to_4th_Grade",
        "B15003_004E": "5th_to_6th_Grade",
        "B15003_005E": "7th_to_8th_Grade",
        "B15003_006E": "9th_Grade",
        "B15003_017E": "High_School_Graduate",
        "B15003_022E": "Bachelor_Degree",
        "B15003_023E": "Master_Degree",
        "B15003_025E": "Doctorate_Degree","state": "State_Code"})

    df_edu["Without Min Studies"]=df_edu[
        ["No_Schooling_Completed",
        "Nursery_to_4th_Grade",
        "5th_to_6th_Grade",
        "7th_to_8th_Grade",
        "9th_Grade"]].sum(axis=1)
    df_edu.drop(columns=["No_Schooling_Completed",
     "Nursery_to_4th_Grade",
     "5th_to_6th_Grade",
     "7th_to_8th_Grade",
     "9th_Grade","State_Code"], inplace=True)
    df_edu["year"]=year
    df_edu = df_edu[[
        "year", "State", "Population_25plus",
        "Without Min Studies", "High_School_Graduate",
        "Bachelor_Degree", "Master_Degree", "Doctorate_Degree"
    ]]
    return df_edu
frame=[]
for year in years:
    print(f"Fetching {year}...")
    df_year = get_edu(year)
    if not df_year.empty:
        frame.append(df_year)
df_edu = pd.concat(frame, ignore_index=True)
df_edu


Fetching 2014...
Fetching 2015...
Fetching 2016...
Fetching 2017...
Fetching 2018...
Fetching 2019...
Fetching 2020...
⚠️ 2020: HTTP 404 – skipping year
Fetching 2021...
Fetching 2022...
Fetching 2023...
Fetching 2024...


,year,State,Population_25plus,Without Min Studies,High_School_Graduate,Bachelor_Degree,Master_Degree,Doctorate_Degree
0,2014,Alabama,3256766,4356549153312303054,847523,478058,201391,33082
1,2014,Alaska,465149,328112814513577,108523,85634,33103,4304
2,2014,Arizona,4436226,6551349773143245977,899108,765784,328400,53578
3,2014,Arkansas,1968414,226383365018461493,568437,272411,104636,19380
4,2014,California,25654292,673107605487393741089646,4768478,5120162,2035257,394265
...,...,...,...,...,...,...,...,...
515,2024,Washington,5631460,958011353291819483343,975478,1372926,683345,113323
516,2024,West Virginia,1268689,13595730139218305,420214,187714,88027,13183
517,2024,Wisconsin,4172638,4413981610241295780,1057997,943201,362471,61628
518,2024,Wyoming,409344,460308100,90634,82918,35122,8475


In [33]:
url_edu = "https://api.census.gov/data/2021/acs/acs1"
params = {
    "get": "NAME,B15003_001E,B15003_002E,B15003_003E,B15003_004E,B15003_005E,"
           "B15003_006E,B15003_017E,B15003_022E,B15003_023E,B15003_025E",
    "for": "state:*"
}
years=list(range(2014,2025))
response_1 = requests.get(url_edu, params=params)

data = response_1.json()
df_edu = pd.DataFrame(data[1:], columns=data[0])
num_cols = [c for c in df_edu.columns if c.startswith("B15003_")]
df_edu[num_cols] = df_edu[num_cols].apply(pd.to_numeric)
df_edu = df_edu.rename(columns={
    "NAME": "State",
    "B15003_001E": "Population_25plus",
    "B15003_002E": "No_Schooling_Completed",
    "B15003_003E": "Nursery_to_4th_Grade",
    "B15003_004E": "5th_to_6th_Grade",
    "B15003_005E": "7th_to_8th_Grade",
    "B15003_006E": "9th_Grade",
    "B15003_017E": "High_School_Graduate",
    "B15003_022E": "Bachelor_Degree",
    "B15003_023E": "Master_Degree",
    "B15003_025E": "Doctorate_Degree","state": "State_Code"
  
})
df_edu["Without Min Studies"]=df_edu[
    ["No_Schooling_Completed",
     "Nursery_to_4th_Grade",
     "5th_to_6th_Grade",
     "7th_to_8th_Grade",
     "9th_Grade"]
].sum(axis=1)
df_edu.drop(columns=["No_Schooling_Completed",
     "Nursery_to_4th_Grade",
     "5th_to_6th_Grade",
     "7th_to_8th_Grade",
     "9th_Grade","State_Code"], inplace=True)

print(df_edu.columns)
df_edu


Index(['State', 'Population_25plus', 'High_School_Graduate', 'Bachelor_Degree',
       'Master_Degree', 'Doctorate_Degree', 'Without Min Studies'],
      dtype='object')


,State,Population_25plus,High_School_Graduate,Bachelor_Degree,Master_Degree,Doctorate_Degree,Without Min Studies
0,Alabama,3451208,902759,572276,271514,45110,57473
1,Puerto Rico,2401409,619884,493356,127459,33486,75728
2,Arizona,4980297,967537,985673,455187,76263,95958
3,Arkansas,2037763,572881,324137,140186,21775,35714
4,California,26909869,4909898,5958030,2573469,511191,1101187
5,Colorado,4044182,658973,1107309,500621,82500,63624
6,Connecticut,2534376,568748,587690,348217,48686,53943
7,Delaware,711104,178620,146295,80771,11932,9764
8,District of Columbia,478774,59776,120771,109362,23602,10602
9,Florida,15762122,3686864,3248221,1368496,225826,327997
